In [57]:
import numpy as np
import osmnx as ox
from osgeo import osr, ogr
import warnings

warnings.filterwarnings('ignore')

place_name = 'Пермь Ленинский район'

In [58]:
geo_df = ox.footprints_from_place(place_name)

geo_df = geo_df[['building', 'building:levels', 'geometry']]
geo_df = geo_df.reset_index(drop=True)

geo_df = calculate_region_data(geo_df)

In [59]:
living_buildings_types = ['apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'ger', 'hotel',
                           'house', 'houseboat', 'residential', 'semidetached_house', 'static_caravan', 'terrace']
real_live_buildings = geo_df.loc[geo_df.building.isin(living_buildings_types)]
living_building_dict = dict(real_live_buildings.building.value_counts())

In [60]:
population, fake_houses_count = calculate_people_count(geo_df, living_buildings_types)
print(f"В {place_name} проживает {round(population)} человек\n"
      f"Разновидности жилых зданий в {place_name}: {living_building_dict}, их количество {sum(living_building_dict.values())}\n"
      f"Предполагаемых жилых зданий в {place_name}: {fake_houses_count}")

Реальных людей посчитано 25502 человек
Предполагаемых людей посчитано 31147 человек
В Пермь Ленинский район проживает 56649 человек
Разновидности жилых зданий в Пермь Ленинский район: {'house': 574, 'apartments': 75, 'dormitory': 15, 'residential': 6, 'hotel': 4}, их количество 674
Предполагаемых жилых зданий в Пермь Ленинский район: 562


In [25]:
def calculate_region_data(geo_df):
    geo_df["area"], geo_df["edges_count"] = None, None
    geo_df.loc[geo_df['building:levels'].isnull(), 'building:levels'] = 1
    for index, element in geo_df.iterrows():
        geo_df["area"][index] = get_area_from_polygon(element.geometry)
        geo_df["edges_count"][index] = get_edges_count(element.geometry)
    geo_df = geo_df.drop(['geometry'], axis=1)
    geo_df = geo_df.reset_index(drop=True)
    geo_df.edges_count = geo_df.edges_count.astype('float')
    geo_df = geo_df[np.isfinite(geo_df['edges_count'])]
    geo_df = geo_df.fillna(0)
    return geo_df

def get_edges_count(geometry_obj):
    if geometry_obj.type == 'MultiPolygon':
        count = 0
        for polygon in geometry_obj:
            count += polygon.exterior.coords.__len__()
    elif geometry_obj.type == 'Polygon':
        count = geometry_obj.exterior.coords.__len__()
    return count

In [7]:
def get_area_from_polygon(geometry_obj):
    if geometry_obj.type == 'MultiPolygon':
        area = 0
        for polygon in geometry_obj:
            area += __calculate_polygon_area(polygon)
    elif geometry_obj.type == 'Polygon':
        area = __calculate_polygon_area(geometry_obj)
    return area

def __calculate_polygon_area(polygon_obj):
    json = {
        'type': 'Polygon',
        'coordinates': [list(polygon_obj.exterior.coords)]
    }

    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)
    target = osr.SpatialReference()
    target.ImportFromEPSG(5243)

    transform = osr.CoordinateTransformation(source, target)
    poly = ogr.CreateGeometryFromJson(str(json).replace('(', '[').replace(')', ']'))
    poly.Transform(transform)
    area = poly.GetArea()
    return area

In [55]:
def calculate_people_count(geo_df, living_buildings_types):
    fake_houses = 0
    real_people_count = 0
    fake_people_count = 0
    for index, row in geo_df.iterrows():
        if row['building'] in living_buildings_types:
            real_people_count += row.area * int(row['building:levels']) / 1.5 / 22
        elif int(row['building:levels']) > 1 and row['building'] == 'yes' and row['edges_count'] < 13:
            fake_people_count += row.area * int(row['building:levels']) / 3 / 22
            fake_houses += 1
    print('Реальных людей посчитано {} человек'.format(round(real_people_count)))
    print('Предполагаемых людей посчитано {} человек'.format(round(fake_people_count)))
    
    return real_people_count + fake_people_count , fake_houses

In [ ]:
place_name = 'Пермь Свердловский район'
dateea = ox.footprints_from_place(place_for_search)

In [95]:
dateea

,nodes,addr:housenumber,addr:housenumber2,addr:street,addr:street2,building,building:levels,geometry,amenity,architect,...,building:material,abandoned,diaper,toilets:disposal,unisex,construction,preschool,members,type,aeroway
24223415,"[262279469, 262279470, 262279471, 1601214377, ...",113,23,Уральская улица,улица Крупской,apartments,5,"POLYGON ((56.27796 58.01913, 56.27816 58.01919...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268090,"[262749035, 262749036, 262749037, 262749038, 2...",25,NaN,улица Крупской,NaN,apartments,5,"POLYGON ((56.27800 58.01906, 56.27864 58.01835...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268098,"[262750118, 262750119, 262750120, 262750121, 2...",27,48,улица Крупской,улица Лебедева,apartments,5,"POLYGON ((56.27871 58.01828, 56.27899 58.01796...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268138,"[262750938, 262750939, 262750940, 262750941, 2...",46,NaN,улица Лебедева,NaN,yes,5,"POLYGON ((56.28031 58.01830, 56.28141 58.01857...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268141,"[262751605, 262751606, 262751607, 262751608, 2...",46А,NaN,улица Лебедева,NaN,kindergarten,2,"POLYGON ((56.27956 58.01860, 56.28003 58.01871...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421905,NaN,NaN,NaN,NaN,NaN,yes,NaN,"POLYGON ((56.25800 57.98074, 56.25799 57.98055...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{167565569: 'outer', 255534998: 'outer'}",multipolygon,NaN
3421908,NaN,NaN,NaN,NaN,NaN,industrial,NaN,"POLYGON ((56.25362 57.96869, 56.25238 57.96872...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{167565573: 'outer', 255534993: 'outer'}",multipolygon,NaN
3436636,NaN,NaN,NaN,NaN,NaN,yes,NaN,"POLYGON ((56.24230 57.99971, 56.24244 57.99975...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{256432858: 'outer'},multipolygon,hangar
4817206,NaN,NaN,NaN,NaN,NaN,yes,2,"POLYGON ((56.27463 57.96646, 56.27421 57.96685...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{339741813: 'outer', 106475983: 'outer'}",multipolygon,NaN


In [111]:
print(dateea['geometry'][6101546].type == 'MultiPolygon')
(dateea['geometry'][4817206].type) == 'Polygon'

True


True